In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [5]:
#Store Part I results into DataFrame
#Load the csv exported in Part I to a DataFrame

# Create variable & csv path to cities
worldcities_to_load = "../WeatherPy/worldcities.csv"
worldcities = pd.read_csv(worldcities_to_load)
worldcities.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [6]:

# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = worldcities[["Lat", "Lng"]]

humidity = worldcities["Humidity"].astype(float)

In [7]:

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Filter vacation with zero cloudiness
vacation_no_cloud = worldcities[worldcities["Cloudiness"] == 0]
# Filter vacation with max temp above 70 degrees F
vacation_above_70_degrees = vacation_no_cloud[vacation_no_cloud["Max Temp"] > 70]
# Filter vacation with max temp below 80 degrees F
vacation_below_80_degrees = vacation_above_70_degrees[vacation_above_70_degrees["Max Temp"] < 80]
# Filter vacation with wind speed below 10 mph
vacation_slow_wind = vacation_below_80_degrees[vacation_below_80_degrees["Wind Speed"] < 10]
# Filter vacation with humidity below 60 %
perfect_vacation = vacation_slow_wind[vacation_slow_wind["Humidity"] < 60]
# Set Index
indexed_perfect_vacation = perfect_vacation.reset_index()
del indexed_perfect_vacation["index"]
indexed_perfect_vacation

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
2,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
3,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
4,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


In [12]:

vaca_locations = indexed_perfect_vacation[["Lat", "Lng"]]

vaca_humidity = indexed_perfect_vacation["Humidity"].astype(float)

# Plot Heatmap
vaca_fig = gmaps.figure()

# Create heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vaca_fig.add_layer(vaca_heat_layer)

# Display figure
vaca_fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(indexed_perfect_vacation["City"])):

    lat = indexed_perfect_vacation.loc[city]["Lat"]
    lng = indexed_perfect_vacation.loc[city]["Lng"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
indexed_perfect_vacation["Nearest Hotel"] = hotels
indexed_perfect_vacation

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Nearest Hotel
0,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,ABBA Guesthouse
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Hotel Escarpas do Lago
2,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel La Giralda
3,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,MOGOK HILL HOTEL
4,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,تاکسی سرویس آرامش


In [16]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in indexed_perfect_vacation.iterrows()]
locations = indexed_perfect_vacation[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
vaca_fig.add_layer(markers)

# Display Map
vaca_fig

Figure(layout=FigureLayout(height='420px'))